# Recommendation system

DB:
- ID;
- Name;
- Rating;
- Episodes;
- Studio;
- Tags;
- Description;
- Related_Mange;
- Recommendations.

RS:
Tags -> Recommendations (list of animes).

In [1]:
import pandas as pd

In [2]:
path_to_csv = "Anime.csv"

df = pd.read_csv(path_to_csv).set_index("Rank")

In [3]:
df.head(3)

,Name,Japanese_name,Type,Episodes,Studio,Release_season,Tags,Rating,Release_year,End_year,Description,Content_Warning,Related_Mange,Related_anime,Voice_actors,staff
Rank,,,,,,,,,,,,,,,,
1,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Kimetsu no Yaiba: Yuukaku-hen,TV,NaN,ufotable,Fall,"Action, Adventure, Fantasy, Shounen, Demons, H...",4.60,2021.0,NaN,'Tanjiro and his friends accompany the Hashira...,Explicit Violence,Demon Slayer: Kimetsu no Yaiba,"Demon Slayer: Kimetsu no Yaiba, Demon Slayer: ...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk...","Koyoharu Gotouge : Original Creator, Haruo Sot..."
2,Fruits Basket the Final Season,Fruits Basket the Final,TV,13.0,TMS Entertainment,Spring,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",4.60,2021.0,NaN,'The final arc of Fruits Basket.',"Emotional Abuse,, Mature Themes,, Physical Abu...","Fruits Basket, Fruits Basket Another","Fruits Basket 1st Season, Fruits Basket 2nd Se...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum...","Natsuki Takaya : Original Creator, Yoshihide I..."
3,Mo Dao Zu Shi 3,The Founder of Diabolism 3,Web,12.0,B.C MAY PICTURES,NaN,"Fantasy, Ancient China, Chinese Animation, Cul...",4.58,2021.0,NaN,'The third season of Mo Dao Zu Shi.',NaN,Grandmaster of Demonic Cultivation: Mo Dao Zu ...,"Mo Dao Zu Shi 2, Mo Dao Zu Shi Q","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang...","Mo Xiang Tong Xiu : Original Creator, Xiong Ke..."


In [4]:
len(df)

18495

In [5]:
df.isnull().sum()

Name                   0
Japanese_name      10557
Type                   0
Episodes            8994
Studio              6477
Release_season     14379
Tags                 400
Rating              3131
Release_year         383
End_year           15641
Description            4
Content_Warning    16655
Related_Mange      10868
Related_anime       8432
Voice_actors        3186
staff               5490
dtype: int64

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors


def process_df_rs(df, columns):
    df = df.copy()
    df = df[columns]
    df = df.dropna()
    return df


def build_recommendations(df, columns=None):
    rs_df = process_df_rs(df, ["Tags"])

    vectorizer = CountVectorizer(token_pattern=r"(?u)(\w[\w ]+)")
    vectors = vectorizer.fit_transform(rs_df.Tags.tolist())

    nbrs = NearestNeighbors(n_neighbors=11).fit(vectors)
    _, indices = nbrs.kneighbors(vectors)
    recommendations = pd.Series(indices.tolist())
    recommendations.name = "recommendations"

    def fix_indices(indices):
        return list(map(lambda idx: rs_df.index[idx], indices))[1:]

    recommendations = recommendations.apply(fix_indices)
    recommendations.index = rs_df.index

    final_df = df.join(recommendations)

    if columns is not None:
        final_df = final_df[columns]

    return final_df


In [41]:
columns = ["Name", "Episodes", "Studio", "Rating", "Description", "Related_Mange", "recommendations"]

final_df = build_recommendations(df, columns=columns)
final_df.index.name = "_id"

In [42]:
def remove_tabs(el):
    return el.replace("\t", " ")

In [43]:
final_df

,Name,Episodes,Studio,Rating,Description,Related_Mange,recommendations
_id,,,,,,,
1,Demon Slayer: Kimetsu no Yaiba - Entertainment...,NaN,ufotable,4.60,'Tanjiro and his friends accompany the Hashira...,Demon Slayer: Kimetsu no Yaiba,"[13, 15011, 3062, 6269, 4422, 7592, 8325, 1013..."
2,Fruits Basket the Final Season,13.0,TMS Entertainment,4.60,'The final arc of Fruits Basket.',"Fruits Basket, Fruits Basket Another","[27, 108, 911, 11948, 5726, 2540, 14694, 4238,..."
3,Mo Dao Zu Shi 3,12.0,B.C MAY PICTURES,4.58,'The third season of Mo Dao Zu Shi.',Grandmaster of Demonic Cultivation: Mo Dao Zu ...,"[3205, 2181, 2235, 3722, 2105, 2379, 2678, 682..."
4,Fullmetal Alchemist: Brotherhood,64.0,Bones,4.58,"""The foundation of alchemy is based on the law...","Fullmetal Alchemist, Fullmetal Alchemist (Ligh...","[274, 1789, 1718, 175, 5853, 14, 525, 15011, 1..."
5,Attack on Titan 3rd Season: Part II,10.0,WIT Studio,4.57,'The battle to retake Wall Maria begins now! W...,"Attack on Titan, Attack on Titan: End of the W...","[78, 43, 828, 417, 88, 3591, 1428, 923, 72, 618]"
...,...,...,...,...,...,...,...
18491,Qin Shi Mingyue: Canghai Hengliu Xiaomeng Spec...,2.0,Sparkly Key Animation Studio,NaN,Special episodes of Qin Shi Mingyue: Canghai H...,NaN,"[18491, 3104, 1826, 3624, 3812, 2860, 2576, 24..."
18492,Yi Tang Juchang: Sanguo Yanyi,108.0,NaN,NaN,No synopsis yet - check back soon!,NaN,"[17558, 17453, 16058, 17483, 16047, 18087, 160..."
18493,Fenghuang Ji Xiang Yu Qingming Shanghe Tu,13.0,NaN,NaN,No synopsis yet - check back soon!,NaN,"[17420, 15492, 18264, 18263, 15497, 17700, 162..."


In [44]:
final_df.to_csv("anime_with_recommendations.csv")

In [4]:
a = '[1, 2]'
a.replace('[', '').replace(']', '').split(',')

['1', ' 2']

In [7]:
import pandas as pd

list(pd.Series([1, 2]).items())

[(0, 1), (1, 2)]